In [1]:
using ODBC
using DataFrames
using DSWB
using Formatting

dsn = "dswb-natgeo" # Redshift esetTable(tableRt, tableType = "RESOURCE_TABLE")ndpoint
table = "beacons_4744" # beacon table name
tableRt = "beacons_4744_rt"

# Connect to Beacon Data
setRedshiftEndpoint(dsn)
setTable(tableRt, tableType = "RESOURCE_TABLE")
setTable(table)

Set Redshift endpoint done. 2017-03-11T15:44:38


"beacons_4744"

In [2]:
# Packages
include("../../Lib/Include-Package-v2.1.jl")

# Time values (tv.) structure created in include above, so init time here
#TV = timeVariables(2017,1,2,2,30,2017,1,2,2,45);
TV = timeVariables(2017,3,8,6,0,2017,3,8,11,59);
#TV = weeklyTimeVariables(days=7);

March 8, 2017 06:00:00 to March 8, 2017 11:59:00 Local Time
March 8, 2017 11:00:00 to March 8, 2017 16:59:00 UTC Time


In [3]:
customer = "Nat Geo" 
productPageGroup = "News Article" # primary page group
localUrl = "http://news.nationalgeographic.com/news/2014/05/140518-dogs-war-canines-soldiers-troops-military-japanese-prisoner/"
localTable = "$(table)_Find_Large_Page_Url_view"
deviceType = "Mobile"
linesOutput = 25
;

In [4]:
try
    query("""\
        create or replace view $localTable as (
            select * from $table 
                where 
                    "timestamp" between $(TV.startTimeMsUTC) and $(TV.endTimeMsUTC) and 
                    page_group ilike '$(productPageGroup)' and
                    params_u ilike '$(localUrl)'  and 
                    $table.user_agent_device_type ilike '$(deviceType)'

        )
    """)
    cnt = query("""SELECT count(*) FROM $localTable""")
    println("$localTable count is ",cnt[1,1])
catch y
    println("setupLocalTable Exception ",y)
end

beacons_4744_Find_Large_Page_Url_view count is 22971


In [32]:
try
    query("""
    select 
        user_agent_device_type,
        user_agent_family,
    user_agent_os,user_agent_manufacturer,params_ua_vnd,
    count(*)
    from $localTable
    group by 
        user_agent_device_type,
        user_agent_family,
    user_agent_os,user_agent_manufacturer,params_ua_vnd
    order by count(*) desc
    limit 30
    """)
catch y
    println("setupLocalTable Exception ",y)
end

,user_agent_device_type,user_agent_family,user_agent_os,user_agent_manufacturer,params_ua_vnd,count
1,Mobile,Mobile Safari UIWebView,iOS,Apple,"Apple Computer, Inc.",7543
2,Mobile,Chrome Mobile,Android OS,Samsung,Google Inc.,6511
3,Mobile,Chrome Mobile,Android OS,NA,Google Inc.,3356
4,Mobile,Mobile Safari,iOS,Apple,"Apple Computer, Inc.",2861
5,Mobile,Chrome Mobile,Android OS,LG,Google Inc.,696
6,Mobile,Chrome Mobile,Android OS,Motorola,Google Inc.,630
7,Mobile,Chrome Mobile,Android OS,OnePlus,Google Inc.,291
8,Mobile,Chrome Mobile,Android OS,HTC,Google Inc.,279
9,Mobile,Chrome Mobile iOS,iOS,Apple,"Apple Computer, Inc.",225
10,Mobile,Facebook,iOS,Apple,"Apple Computer, Inc.",92


In [25]:
try
    query("""
    select 
        user_agent_device_type,
        user_agent_family,
    geo_netspeed, params_cpu_cnc,user_agent_os,user_agent_osversion,user_agent_manufacturer,params_ua_plt,params_ua_vnd,
    count(*)
    from $localTable
    group by 
        user_agent_device_type,
        user_agent_family,
    geo_netspeed, params_cpu_cnc,user_agent_os,user_agent_osversion,user_agent_manufacturer,params_ua_plt,params_ua_vnd
    order by count(*) desc
    limit 25
    """)
catch y
    println("setupLocalTable Exception ",y)
end

,user_agent_device_type,user_agent_family,geo_netspeed,params_cpu_cnc,user_agent_os,user_agent_osversion,user_agent_manufacturer,params_ua_plt,params_ua_vnd,count
1,Mobile,Mobile Safari UIWebView,Cable/DSL,NA,iOS,10,Apple,iPhone,"Apple Computer, Inc.",4074
2,Mobile,Mobile Safari UIWebView,Cellular,NA,iOS,10,Apple,iPhone,"Apple Computer, Inc.",1960
3,Mobile,Mobile Safari,Cable/DSL,NA,iOS,10,Apple,iPhone,"Apple Computer, Inc.",1288
4,Mobile,Chrome Mobile,Cable/DSL,8,Android OS,6,Samsung,Linux armv8l,Google Inc.,1266
5,Mobile,Mobile Safari,Cellular,NA,iOS,10,Apple,iPhone,"Apple Computer, Inc.",675
6,Mobile,Chrome Mobile,Cellular,8,Android OS,6,Samsung,Linux armv8l,Google Inc.,668
7,Mobile,Chrome Mobile,Cable/DSL,4,Android OS,6,Samsung,Linux armv7l,Google Inc.,598
8,Mobile,Chrome Mobile,Cable/DSL,4,Android OS,6,Samsung,Linux armv8l,Google Inc.,458
9,Mobile,Mobile Safari UIWebView,NA,NA,iOS,10,Apple,iPhone,"Apple Computer, Inc.",432
10,Mobile,Mobile Safari UIWebView,Corporate,NA,iOS,10,Apple,iPhone,"Apple Computer, Inc.",393


In [12]:
try
    query("""
    select 
        user_agent_device_type,
        user_agent_family,
        params_scr_xy,
    geo_cc, geo_netspeed, params_cpu_cnc,params_scr_bpp,params_scr_dpx,params_scr_mtp,params_scr_orn,user_agent_os,user_agent_osversion,user_agent_manufacturer,params_ua_plt,params_ua_vnd,
    count(*)
    from $localTable
    group by 
        user_agent_device_type,
        user_agent_family,
        params_scr_xy,
    geo_cc, geo_netspeed, params_cpu_cnc,params_scr_bpp,params_scr_dpx,params_scr_mtp,params_scr_orn,user_agent_os,user_agent_osversion,user_agent_manufacturer,params_ua_plt,params_ua_vnd
    order by count(*) desc
    limit 25
    """)
catch y
    println("setupLocalTable Exception ",y)
end

,user_agent_device_type,user_agent_family,params_scr_xy,geo_cc,geo_netspeed,params_cpu_cnc,params_scr_bpp,params_scr_dpx,params_scr_mtp,params_scr_orn,user_agent_os,user_agent_osversion,user_agent_manufacturer,params_ua_plt,params_ua_vnd,count
1,Mobile,Mobile Safari UIWebView,375x667,US,Cable/DSL,NA,32/32,2.0,NA,NA,iOS,10,Apple,iPhone,"Apple Computer, Inc.",1891
2,Mobile,Mobile Safari UIWebView,375x667,US,Cellular,NA,32/32,2.0,NA,NA,iOS,10,Apple,iPhone,"Apple Computer, Inc.",904
3,Mobile,Mobile Safari UIWebView,414x736,US,Cable/DSL,NA,32/32,3.0,NA,NA,iOS,10,Apple,iPhone,"Apple Computer, Inc.",677
4,Mobile,Mobile Safari,375x667,US,Cable/DSL,NA,32/32,2.0,NA,NA,iOS,10,Apple,iPhone,"Apple Computer, Inc.",593
5,Mobile,Mobile Safari UIWebView,320x568,US,Cable/DSL,NA,32/32,2.0,NA,NA,iOS,10,Apple,iPhone,"Apple Computer, Inc.",526
6,Mobile,Chrome Mobile,360x640,US,Cable/DSL,8,32/32,4.0,5,0/portrait-primary,Android OS,6,Samsung,Linux armv8l,Google Inc.,461
7,Mobile,Chrome Mobile,360x640,US,Cable/DSL,4,32/32,4.0,5,0/portrait-primary,Android OS,6,Samsung,Linux armv8l,Google Inc.,409
8,Mobile,Mobile Safari UIWebView,414x736,US,Cellular,NA,32/32,3.0,NA,NA,iOS,10,Apple,iPhone,"Apple Computer, Inc.",381
9,Mobile,Mobile Safari,375x667,US,Cellular,NA,32/32,2.0,NA,NA,iOS,10,Apple,iPhone,"Apple Computer, Inc.",322
10,Mobile,Mobile Safari UIWebView,320x568,US,Cellular,NA,32/32,2.0,NA,NA,iOS,10,Apple,iPhone,"Apple Computer, Inc.",289


In [13]:
try
    query("""SELECT * FROM $localTable limit 3""")
catch y
    println("setupLocalTable Exception ",y)
end

,compression_types,custom_metrics_0,custom_metrics_1,custom_metrics_2,custom_metrics_3,custom_metrics_4,custom_metrics_5,custom_metrics_6,custom_metrics_7,custom_metrics_8,custom_metrics_9,domain,geo_cc,geo_city,geo_lat,geo_lon,geo_netspeed,geo_org,geo_postalcode,geo_rg,http_method,http_referrer,http_version,key,mobile_connection_type,page_group,params_bat_lvl,params_cmet_mpulseid,params_cpu_cnc,params_custom0_st,params_custom1_st,params_custom2_st,params_custom3_st,params_custom4_st,params_custom5_st,params_custom6_st,params_custom7_st,params_custom8_st,params_custom9_st,params_dom_doms,params_dom_img,params_dom_ln,params_dom_res,params_dom_res_slowest,params_dom_script,params_dom_sz,params_errors,params_h_t,params_if,params_mem_total,params_mem_used,params_mob_bw,params_mob_mt,params_nt_con_end,params_nt_con_st,params_nt_dns_end,params_nt_dns_st,params_nt_domcomp,params_nt_domcontloaded_e,params_nt_domcontloaded_st,params_nt_domint,params_nt_domloading,params_nt_fet_st,params_nt_first_paint,params_nt_load_end,params_nt_load_st,params_nt_nav_st,params_nt_nav_type,params_nt_red_cnt,params_nt_red_end,params_nt_red_st,params_nt_req_st,params_nt_res_end,params_nt_res_st,params_nt_spdy,params_nt_ssl_st,params_nt_unload_end,params_nt_unload_st,params_nu,params_pgu,params_r,params_r2,params_rt_abld,params_rt_blstart,params_rt_bmr_conen,params_rt_bmr_const,params_rt_bmr_domen,params_rt_bmr_domst,params_rt_bmr_fetst,params_rt_bmr_reqst,params_rt_bmr_resen,params_rt_bmr_resst,params_rt_bmr_secst,params_rt_bstart,params_rt_cnf_conen,params_rt_cnf_const,params_rt_cnf_domen,params_rt_cnf_domst,params_rt_cnf_fetst,params_rt_cnf_reqst,params_rt_cnf_resen,params_rt_cnf_resst,params_rt_cnf_secst,params_rt_cstart,params_rt_end,params_rt_ntvu,params_rt_obo,params_rt_quit,params_rt_sh,params_rt_si,params_rt_sl,params_rt_srst,params_rt_start,params_rt_tstart,params_rt_tt,params_scr_bpp,params_scr_dpx,params_scr_mtp,params_scr_orn,params_scr_xy,params_u,params_v,params_vis_lh,params_vis_lv,params_vis_st,proxy_address,remote_ip,session_id,session_isunload,session_latest,session_obopages,session_pages,session_start,session_totalloadtime,spdy,ssl,timers_before_dns,timers_boomerang,timers_boomr_fb,timers_boomr_lat,timers_boomr_ld,timers_boomr_to_end,timers_custom0,timers_custom1,timers_custom2,timers_custom3,timers_custom4,timers_custom5,timers_custom6,timers_custom7,timers_custom8,timers_custom9,timers_dns,timers_domload,timers_domready,timers_fb_to_boomr,timers_loaded,timers_missing,timers_navst_to_boomr,timers_renderstart,timers_ssl,timers_t_configfb,timers_t_configjs,timers_t_domloaded,timers_t_done,timers_t_load,timers_t_page,timers_t_postrender,timers_t_prerender,timers_t_resp,timers_tcp,timestamp,url,user_agent_family,user_agent_major,user_agent_minor,user_agent_mobile,user_agent_model,user_agent_os,user_agent_osversion,user_agent_raw,warnings,beacon_type,site_version,ab_test,ipv6,user_agent_manufacturer,user_agent_device_type,user_agent_isp,geo_isp,bandwidth_kbps,bandwidth_error_pc,bandwidth_block,headers_cf_visitor,headers_cf_ray,headers_cf_connecting_ip,headers_x_forwarded_for,headers_x_forwarded_proto,headers_connection,headers_host,headers_accept_encoding,headers_accept_language,headers_accept,headers_content_length,headers_various,params_dom_img_ext,params_dom_script_ext,params_ua_plt,params_ua_vnd,http_errno,params_http_method,params_http_hdr,params_http_initiator,params_h_cr,params_xhr_sync,params_rt_subres,params_bw_time,params_lat,params_lat_err,params_mob_ct,cdim,params_nt_cinf,params_rt_ss,errors,tp_ga_clientid,tp_ga_utm_source,tp_ga_utm_medium,tp_ga_utm_term,tp_ga_utm_content,tp_ga_utm_campaign,tp_aa_aid,tp_aa_mid,tp_aa_campaign,tp_ia_coreid,tp_ia_mmc_vendor,tp_ia_mmc_category,tp_ia_mmc_placement,tp_ia_mmc_item,tp_ia_sp_type,tp_ia_sp_promotion,tp_ia_sp_link,tp_ia_re_version,tp_ia_re_pagearea,tp_ia_re_link
1,NA,1,NA,NA,NA,NA,NA,NA,NA,NA,NA,nationalgeographic.com,GB,NA,51.5,-0.13000488,Cellular,Vodafone Limited,NA,NA,POST,http://news

In [19]:
joinTablesTest = DataFrame()

try
    joinTablesTest = query("""\
    select $tableRt.*
    from $localTable join $tableRt
    on $localTable.session_id = $tableRt.session_id and $localTable."timestamp" = $tableRt."timestamp"
    where $tableRt.encoded_size > 1
    limit 3
    """);

    beautifyDF(joinTablesTest[1:min(linesOutput,end),:])
catch y
    println("bigTable5 Exception ",y)
end
#display(joinTables)

params u,session id,session start,timestamp,url,initiator type,start time,redirect start,redirect end,fetch start,dns start,dns end,tcp connection start,secure connection start,tcp connection end,request start,response first byte,response last byte,worker start,encoded size,transferred size,decoded size,height,width,x,y
http://news.nationalgeographic.com/news/2014/05/140518-dogs-war-canines-soldiers-troops-military-japanese-prisoner/,e1affb77-1caf-4d29-804b-4d9924d03159-omhvuy,"1,488,974,218,253",2017-03-08T11:57:08.653,https://securepubads.g.doubleclick.net/gampad/ads?...,script,"7,101",0,0,"7,101",0,0,0,0,0,"7,198","7,296","7,300",NA,"3,599","3,658","8,507",NA,NA,NA,NA
http://news.nationalgeographic.com/news/2014/05/140518-dogs-war-canines-soldiers-troops-military-japanese-prisoner/,e1affb77-1caf-4d29-804b-4d9924d03159-omhvuy,"1,488,974,218,253",2017-03-08T11:57:08.653,https://securepubads.g.doubleclick.net/gpt/pubads_impl_111.js,script,"6,581",0,0,"6,581",0,0,0,0,0,"6,583","6,594","6,603",NA,"61,267",0,"179,089",NA,NA,NA,NA
http://news.nationalgeographic.com/news/2014/05/140518-dogs-war-canines-soldiers-troops-military-japanese-prisoner/,e1affb77-1caf-4d29-804b-4d9924d03159-omhvuy,"1,488,974,218,253",2017-03-08T11:57:08.653,http://news.nationalgeographic.com/etc/designs/news/scripts/init-v2.ngsversion.99ead940.js,script,349,0,0,349,0,0,0,0,0,355,388,390,NA,"1,778","2,211","2,997",NA,NA,NA,NA


scrubUrlToPrint Exception KeyError(:urlgroup)


In [21]:
joinTables = DataFrame()

try
    joinTables = query("""\
    select 
        $tableRt.url as urlgroup,
        $localTable.user_agent_device_type,
        $localTable.user_agent_family as useragentfamily,
        $localTable.params_scr_xy,
        $localTable.session_id,
        $localTable."timestamp",
        sum($tableRt.encoded_size) as encoded,
        sum($tableRt.transferred_size) as transferred,
        sum($tableRt.decoded_size) as decoded,
        count(*)
    from $localTable join $tableRt
    on $localTable.session_id = $tableRt.session_id and $localTable."timestamp" = $tableRt."timestamp"
    where $tableRt.encoded_size > 1
    group by $tableRt.url,$localTable.user_agent_device_type,$localTable.user_agent_family,
        $localTable.params_scr_xy,$localTable.session_id,$localTable."timestamp"
    order by encoded desc
    """);

    scrubUrlToPrint(joinTables,:urlgroup,limit=150)
    beautifyDF(joinTables[1:min(linesOutput,end),:])
catch y
    println("bigTable5 Exception ",y)
end
#display(joinTables)

urlgroup,user agent device type,useragentfamily,params scr xy,session id,timestamp,encoded,transferred,decoded,count
http://http.atlas.cdn.yimg.com/yamplus/video_4Kpn-UN2D5UrcaY2rMLRvYJCkNYx-kmzPTzHtNBpwWOJUIqzOQSLHW-lMtgxKxmq8iwbj9wuQNU-_12.mp4?a=yamplus_mr=0_c=5478...,Mobile,Chrome Mobile,346x615,47ee2851-20d9-4982-aead-55382439df6b-omi2f3,2017-03-08T14:28:00.568,"2,076,308","2,076,753",NA,1
http://news.nationalgeographic.com/etc/designs/platform/v2/scripts/ngs-vendor.ngsversion.99ead940.js,Mobile,Chrome Mobile,360x640,7c17513e-eabc-4850-9982-685f67461a18-omi0tr,2017-03-08T13:44:28.207,"882,783","884,844",NA,1
http://news.nationalgeographic.com/etc/designs/platform/v2/scripts/ngs-vendor.ngsversion.99ead940.js,Mobile,Chrome Mobile,360x640,7c17513e-eabc-4850-9982-685f67461a18-omi798,2017-03-08T16:03:34.600,"882,783",0,NA,1
http://news.nationalgeographic.com/etc/designs/platform/v2/scripts/ngs-vendor.ngsversion.99ead940.js,Mobile,Chrome Mobile,360x640,ccb05af4-37da-415e-86cc-2552a2a8c688-omi4xs,2017-03-08T15:13:37.668,"882,783","883,994",NA,1
http://news.nationalgeographic.com/etc/designs/platform/v2/scripts/ngs-vendor.ngsversion.99ead940.js,Mobile,Chrome Mobile,360x640,8a77a47d-3282-412e-bdaf-2a023e077e59-omi2ww,2017-03-08T14:29:43.914,"882,783","883,701",NA,1
http://news.nationalgeographic.com/etc/designs/platform/v2/scripts/ngs-vendor.ngsversion.99ead940.js,Mobile,Chrome Mobile,360x640,a68b5494-32c5-4172-9fd8-d8d62cef1e0b-omi4e0,2017-03-08T15:02:00.784,"882,783","883,304",NA,1
http://news.nationalgeographic.com/etc/designs/platform/v2/scripts/ngs-vendor.ngsversion.99ead940.js,Mobile,Chrome Mobile,360x640,ad05e5ee-0887-4a16-ad65-89e1c4ea0af2-omi0l4,2017-03-08T13:38:11.368,"882,783","883,189",NA,1
http://news.nationalgeographic.com/etc/designs/platform/v2/scripts/ngs-vendor.ngsversion.99ead940.js,Mobile,Chrome Mobile,360x640,b7c8c5ba-82d4-4330-aaa4-5db743dc9a87-omi8wb,2017-03-08T16:38:44.605,"882,783","883,321",NA,1
http://news.nationalgeographic.com/etc/designs/platform/v2/scripts/ngs-vendor.ngsversion.99ead940.js,Mobile,Chrome Mobile,412x732,e4a8e8b9-3b66-49ce-8612-f71a18562fba-omi7xc,2017-03-08T16:18:04.584,"882,783","883,249",NA,1
http://news.nationalgeographic.com/etc/designs/platform/v2/scripts/ngs-vendor.ngsversion.99ead940.js,Mobile,Chrome Mobile,360x640,f1e48a0b-6e91-4c6d-96ec-85b3b47738b2-omi4tg,2017-03-08T15:11:03.315,"882,783","883,310",NA,1


In [17]:
#topSessionId = joinTables[1:1,:session_id]
#topTimeStamp = joinTables[1:1,:timestamp]
#println("tsi=",topSessionId," tts=", topTimeStamp)

joinTableSummary = DataFrame()
joinTableSummary[:useragentfamily] = "delete"
joinTableSummary[:session_id] = ""
joinTableSummary[:timestamp] = 0
joinTableSummary[:encoded] = 0
joinTableSummary[:transferred] = 0
joinTableSummary[:decoded] = 0
joinTableSummary[:count] = 0

sort!(joinTables,cols=[order(:encoded,rev=true)])
for subDf in groupby(joinTables,:useragentfamily)
    #beautifyDF(subDf[1:1,:])
    i = 1
    for row in eachrow(subDf)
        if (i == 1) 
            i +=1
            push!(joinTableSummary,[row[:useragentfamily],row[:session_id],row[:timestamp],row[:encoded],row[:transferred],row[:decoded],row[:count]])
        end            
    end
end

i = 1
for x in joinTableSummary[:,:useragentfamily]
    if x == "delete"
        deleterows!(joinTableSummary,i)
    end
    i += 1
end
sort!(joinTableSummary,cols=[order(:encoded,rev=true)])
;


In [18]:
beautifyDF(joinTableSummary[1:min(linesOutput,end),[:useragentfamily,:encoded,:transferred,:decoded]])

useragentfamily,encoded,transferred,decoded
Chrome Mobile,"3,517,188","3,215,096","802,816"
Firefox Mobile,"2,414,440","2,442,458","3,428,260"
Facebook,"1,592,960","1,534,312","3,065,894"
Opera Mobile,"1,383,170","1,284,023","2,899,879"
Chrome,"1,243,132","1,158,045","2,726,701"
Amazon Silk,"1,121,250","986,528","3,226,817"


In [8]:
function detailsPrint(localTable::ASCIIString,tableRt::ASCIIString,joinTableSummary::DataFrame,row::Int64)
    try
        topSessionId = joinTableSummary[row:row,:session_id][1]
        topTimeStamp = joinTableSummary[row:row,:timestamp][1]
        topTitle = joinTableSummary[row:row,:urlgroup][1]

        joinTablesDetails = query("""\
        select 
        $tableRt.start_time,
        $tableRt.encoded_size,
        $tableRt.transferred_size,
        $tableRt.decoded_size,
        $tableRt.url as urlgroup
        from $localTable join $tableRt
        on $localTable.session_id = $tableRt.session_id and $localTable."timestamp" = $tableRt."timestamp"
        where 
        $localTable.session_id = '$(topSessionId)' and
        $localTable."timestamp" = $(topTimeStamp) and 
        $tableRt.encoded_size > 1000000
        order by $tableRt.start_time
        """);

        displayTitle(chart_title = "Large Requests for: $(topTitle)", chart_info = [TV.timeString], showTimeStamp=false)
        scrubUrlToPrint(joinTablesDetails,:urlgroup,limit=250)
        beautifyDF(joinTablesDetails[1:end,:])
    catch y
        println("bigTable5 Exception ",y)
    end
end


detailsPrint (generic function with 1 method)

In [9]:
function statsTableDF2(table::ASCIIString,productPageGroup::ASCIIString,localUrl::ASCIIString,deviceType::ASCIIString,startTimeMs::Int64, endTimeMs::Int64)
    try
        #println(localUrl)
        
        localStats = query("""\
        select timers_t_done from $table where 
        page_group ilike '$(productPageGroup)' and
        params_u ilike '$(localUrl)' and
        user_agent_device_type ilike '$(deviceType)' and        
        "timestamp" between $startTimeMs and $endTimeMs and
        params_rt_quit IS NULL
        """);
        return localStats
    catch y
        println("statsTableDF Exception ",y)
    end
end



statsTableDF2 (generic function with 1 method)

In [10]:


function statsDetailsPrint2(localTable::ASCIIString,tableRt::ASCIIString,joinTableSummary::DataFrame,row::Int64)
    try
        topUrl = string(joinTableSummary[row:row,:urlgroup][1],"%")
        topTitle = joinTableSummary[row:row,:urlgroup][1]
        
        dispDMT = DataFrame(RefGroup=["","",""],Unit=["","",""],Count=[0,0,0],Mean=[0.0,0.0,0.0],Median=[0.0,0.0,0.0],Min=[0.0,0.0,0.0],Max=[0.0,0.0,0.0])
        
        statsFullDF2 = statsTableDF2(localTable,productPageGroup,topUrl,"Desktop",TV.startTimeMsUTC,TV.endTimeMsUTC)
        dispDMT[1:1,:RefGroup] = "Desktop"
        if (size(statsFullDF2)[1] > 0)
            statsDF2 = basicStats(statsFullDF2,productPageGroup,TV.startTimeMsUTC,TV.endTimeMsUTC)
            dispDMT[1:1,:Unit] = statsDF2[2:2,:unit]
            dispDMT[1:1,:Count] = statsDF2[2:2,:count]
            dispDMT[1:1,:Mean] = statsDF2[2:2,:mean]
            dispDMT[1:1,:Median] = statsDF2[2:2,:median]
            dispDMT[1:1,:Min] = statsDF2[2:2,:min]
            dispDMT[1:1,:Max] = statsDF2[2:2,:max]
        end
        statsFullDF2 = statsTableDF2(localTable,productPageGroup,topUrl,"Mobile",TV.startTimeMsUTC,TV.endTimeMsUTC)
        dispDMT[2:2,:RefGroup] = "Mobile"
        if (size(statsFullDF2)[1] > 0)
            statsDF2 = basicStats(statsFullDF2,productPageGroup,TV.startTimeMsUTC,TV.endTimeMsUTC)
            dispDMT[2:2,:Unit] = statsDF2[2:2,:unit]
            dispDMT[2:2,:Count] = statsDF2[2:2,:count]
            dispDMT[2:2,:Mean] = statsDF2[2:2,:mean]
            dispDMT[2:2,:Median] = statsDF2[2:2,:median]
            dispDMT[2:2,:Min] = statsDF2[2:2,:min]
            dispDMT[2:2,:Max] = statsDF2[2:2,:max]
        end
        statsFullDF2 = statsTableDF2(localTable,productPageGroup,topUrl,"Tablet",TV.startTimeMsUTC,TV.endTimeMsUTC)
        dispDMT[3:3,:RefGroup] = "Tablet"
        if (size(statsFullDF2)[1] > 0)
            statsDF2 = basicStats(statsFullDF2,productPageGroup,TV.startTimeMsUTC,TV.endTimeMsUTC)
            dispDMT[3:3,:Unit] = statsDF2[2:2,:unit]
            dispDMT[3:3,:Count] = statsDF2[2:2,:count]
            dispDMT[3:3,:Mean] = statsDF2[2:2,:mean]
            dispDMT[3:3,:Median] = statsDF2[2:2,:median]
            dispDMT[3:3,:Min] = statsDF2[2:2,:min]
            dispDMT[3:3,:Max] = statsDF2[2:2,:max]
        end

        displayTitle(chart_title = "Large Request Stats for: $(topTitle)", chart_info = [TV.timeString], showTimeStamp=false)
        beautifyDF(dispDMT)
    catch y
        println("statsTableDF2 Exception ",y)
    end
end


statsDetailsPrint2 (generic function with 1 method)

In [11]:
i = 0
for row in eachrow(joinTableSummary)
    i += 1
    detailsPrint(localTable,tableRt,joinTableSummary,i)
    statsDetailsPrint2(localTable,tableRt,joinTableSummary,i)
    if (i >= linesOutput)
        break;
    end
end
;

Large Requests for: http://news.nationalgeographic.com/news/2014/05/140518-dogs-war-canines-soldiers-troops-military-japanese-prisoner March 8, 2017 06:00:00 to March 8, 2017 11:59:00 Local Time

Large Request Stats for: http://news.nationalgeographic.com/news/2014/05/140518-dogs-war-canines-soldiers-troops-military-japanese-prisoner March 8, 2017 06:00:00 to March 8, 2017 11:59:00 Local Time

RefGroup,Unit,Count,Mean,Median,Min,Max
Desktop,,0,0,0,0,0
Mobile,seconds,"21,668",13.31,8.385,0.268,"4,731.882"
Tablet,,0,0,0,0,0


scrubUrlToPrint Exception KeyError(:urlgroup)
